### Loading packages, constants, and functions

In [40]:
import functions as f
import constants as c
import classes as cl

import pandas as pd
import os, joblib
print(os.listdir())

['mod_data', 'functions.py', 'Notebook.ipynb', 'constants.py', 'raw_data', '__pycache__', 'README.md', 'classes.py', '.git']


In [28]:
long = c.cwd + "\\raw_data\\long_alzheimers.tsv"
short = c.cwd + "\\raw_data\\short_alzheimers.tsv"


#### Loading datasets using pandas

In [46]:
chunksize = 10
info_df = pd.read_csv("https://stringdb-downloads.org/download/protein.info.v12.0/9606.protein.info.v12.0.txt.gz", compression='gzip', sep="\t")
info_df = info_df[["#string_protein_id"]]
id_prot = info_df.to_dict()
joblib.dump(id_prot, "./joblib_vars/id_prot.joblib")

['./joblib_vars/id_prot.joblib']

In [53]:
"""
An example of an interaction dataset loaded
"""
alz_int_df = pd.read_csv("./raw_data/long_alzheimers.tsv", sep="\t")
alz_int_df.columns = ["Prot1_ShortName", "Prot2_ShortName", "protein1", "protein2", "col1", "col2", "col3", "col4", "col5", "col6", "col7", "col8", "col9"]
alz_int_df

,Prot1_ShortName,Prot2_ShortName,protein1,protein2,col1,col2,col3,col4,col5,col6,col7,col8,col9
0,A2M,APOA1,9606.ENSP00000323929,9606.ENSP00000236850,0.0,0.0,0.0,0.0,0.111,0.000,0.75,0.994,0.998
1,A2M,LRP1,9606.ENSP00000323929,9606.ENSP00000243077,0.0,0.0,0.0,0.0,0.132,0.504,0.00,0.997,0.998
2,A2M,APOE,9606.ENSP00000323929,9606.ENSP00000252486,0.0,0.0,0.0,0.0,0.270,0.292,0.00,0.957,0.975
3,A2M,SORL1,9606.ENSP00000323929,9606.ENSP00000260197,0.0,0.0,0.0,0.0,0.000,0.066,0.00,0.574,0.585
4,A2M,ADAM10,9606.ENSP00000323929,9606.ENSP00000260408,0.0,0.0,0.0,0.0,0.000,0.000,0.00,0.498,0.497
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1354,TOMM40,VDAC1,9606.ENSP00000410339,9606.ENSP00000378487,0.0,0.0,0.0,0.0,0.063,0.428,0.75,0.749,0.962
1355,TREM2,UNC5C,9606.ENSP00000362205,9606.ENSP00000406022,0.0,0.0,0.0,0.0,0.000,0.000,0.00,0.495,0.495
1356,TREM2,ZCWPW1,9606.ENSP00000362205,9606.ENSP00000381109,0.0,0.0,0.0,0.0,0.000,0.000,0.00,0.514,0.514
1357,VDAC1,VPS35,9606.ENSP00000378487,9606.ENSP00000299138,0.0,0.0,0.0,0.0,0.065,0.292,0.00,0.260,0.467


In [ ]:
Simple_int_df = 